In [36]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


In [37]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [39]:
le=LabelEncoder()
data['Gender']=le.fit_transform(data['Gender'])

In [40]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
ohe_encoder=ohe.fit_transform(data[['Geography']])
ohe_df=pd.DataFrame(ohe_encoder.toarray(),columns=ohe.get_feature_names_out())

In [41]:
data=pd.concat([data.drop('Geography',axis=1),ohe_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [42]:
X = data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [43]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [44]:
with open('label_encoder.pkl','wb') as file:
    pickle.dump(le,file)

with open('ohe_encoder.pkl','wb') as file:
    pickle.dump(ohe,file)


with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [46]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])


model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [48]:
log_dir="logs/fits" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensoflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
early_stopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


In [49]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensoflow_callback,early_stopping])

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 100354.3281 - mae: 100354.3281 - val_loss: 98430.6641 - val_mae: 98430.6641
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99317.9766 - mae: 99317.9766 - val_loss: 96369.8047 - val_mae: 96369.8047
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 95777.8594 - mae: 95777.8594 - val_loss: 91260.2266 - val_mae: 91260.2266
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 89023.2344 - mae: 89023.2344 - val_loss: 83018.4375 - val_mae: 83018.4375
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 79590.1562 - mae: 79590.1562 - val_loss: 73055.5703 - val_mae: 73055.5703
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 69396.4531 - mae: 69396.4531 - val_loss: 63667.0117 - val_mae: 63667.0117
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 60559.1680 - mae: 60559.1680 

In [50]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE:{test_mae}')

63/63 [==============================] - 0s 2ms/step - loss: 50191.7383 - mae: 50191.7383
Test MAE:50191.73828125


In [51]:
model.save('akash.h5')

c:\Users\akash\OneDrive\Desktop\ANN ClASSIFICATION\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
